In [ ]:
#导入库
import numpy as np
from gensim.models import Word2Vec
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score
import pandas as pd


In [ ]:
#读取数据
train_path=r"data/train_set/train_set.csv"
# test_path=r""
df_train=pd.read_csv(train_path,sep="\t", nrows=50000)
# df_test=pd.read_csv(test_path,sep="\t")

#构造长度属性
df_train["len"]=df_train["text"].apply(lambda x:len(x.split(" ")))
# df_train.head()
df_train["len"].describe()
#转换为list
text=df_train["text"].apply(lambda x:x.split(" ")).tolist()

#使用word2vec来训练词向量
model= Word2Vec(text, size=100, window=5, min_count=1, workers=4,sg=0,cbow_mean=1)
model.save("word2vec.model")

max_len=300
x=np.zeros((len(text),max_len,100))#数据量为50000，每个文档有300个词，每个词的向量表达为100
print(x.shape)
for i,sentence in enumerate(text):
#     final=np.zeros(300)
#     m=max(max_len,len(sentence))
    for j in range(max_len):
        if j<len(sentence):
            x[i][j]=x[i][j]+model.wv[sentence[j]]
        else:
            break
#     x[i]=x[i]/df_train["len"][i]
print(x.shape)

#重构特征集形状
x.reshape(-1,max_len,100,1)
print(x.shape)


In [ ]:
#划分数据集跟测试集
from keras.utils import np_utils

x_train=x[:40000]
x_test=x[40000:]
y_train=df_train["label"][:40000]
y_test=df_train["label"][40000:]
y_train=np_utils.to_categorical(y_train,num_classes=14)#用onehot格式表示label
y_test=np_utils.to_categorical(y_test,num_classes=14)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Convolution2D,MaxPooling2D,Flatten
from keras.optimizers import Adam

#初始化模型
cnn=Sequential()
#第一个卷积层
cnn.add(Convolution2D(
input_shape=(300,100,1),
filters=32,
kernel_size=5,
strides=1,
padding="same",
activation="relu"))
#第一个池化层
cnn.add(MaxPooling2D(
pool_size=2,
strides=2,
padding="same"
))
#第二个卷积层
cnn.add(Convolution2D(
filters=64,
kernel_size=5,
strides=1,
padding="same",
activation="relu"))
#第二个池化层
cnn.add(MaxPooling2D(2,2,"same"))
#把第二个卷积层的输出扁平化为一维
cnn.add(Flatten())

#第一个全连接层
cnn.add(Dense(1024,activation="relu"))
#dropout
cnn.add(Dropout(0.4))
#第一个全连接层
cnn.add(Dense(14,activation="softmax"))

#定义优化器
adam=Adam(lr=1e-3)

cnn.complie(optimizer=adam,loss="categorical_crossentropy",metrics=["f1_score"])

cnn.fit(x_train,y_train,batch_size=100,epoch=10)
loss,f1_score=cnn.evaluate(x_test,y_test)

print(f1_score)